# Download and process sentinel 2 data

## John Brandt
## December 2, 2020

## Package imports, API import, source scripts

In [5]:
import datetime
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import scipy.sparse as sparse
import seaborn as sns
import yaml

from collections import Counter
from osgeo import ogr, osr
from random import shuffle
from scipy.sparse.linalg import splu
from sentinelhub import WmsRequest, WcsRequest, MimeType
from sentinelhub import CRS, BBox, constants, DataSource, CustomUrlParam
from skimage.transform import resize
from typing import Tuple, List

with open("../config.yaml", 'r') as stream:
        key = (yaml.safe_load(stream))
        API_KEY = key['key'] 
        
%matplotlib inline
%run ../src/preprocessing/slope.py
%run ../src/preprocessing/indices.py
%run ../src/downloading/utils.py
%run ../src/preprocessing/cloud_removal.py
%run ../src/preprocessing/whittaker_smoother.py
%run ../src/dsen2/utils/DSen2Net.py

In [19]:
if os.path.exists("../config.yaml"):
    with open("../config.yaml", 'r') as stream:
        key = (yaml.safe_load(stream))
        API_KEY = key['key']
        AWSKEY = key['awskey']
        AWSSECRET = key['awssecret']
else:
    API_KEY = "none"

## Parameters

In [6]:
# Parameters
YEAR = 2019
TIME = ('{}-12-01'.format(str(YEAR - 1)), '{}-02-01'.format(str(YEAR + 1)))
EPSG = CRS.WGS84
IMSIZE = 48

# Constants
starting_days = np.cumsum([0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30])

# Bounding boxes

In [8]:
def calc_bbox(plot_id: int, df: "DataFrame") -> List:
    """ Calculates the corners of a bounding box from an input
        pandas dataframe as output by Collect Earth Online

        Parameters:
         plot_id (int): plot_id of associated plot
         df (pandas.DataFrame): dataframe of associated CEO survey
    
        Returns:
         bounding_box (list): [(min(x), min(y)),
                              (max(x), max_y))]
    """
    subs = df[df['PLOT_ID'] == plot_id]
    # (min x, min y), (max x, max y)
    return [(min(subs['LON']), min(subs['LAT'])),
            (max(subs['LON']), max(subs['LAT']))]

def bounding_box(points: List[Tuple[float, float]], 
                 expansion: int = 160) -> ((Tuple, Tuple), str):
    """ Calculates the corners of a bounding box with an
        input expansion in meters from a given bounding_box
        
        Subcalls:
         calculate_epsg, convertCoords

        Parameters:
         points (list): output of calc_bbox
         expansion (float): number of meters to expand or shrink the
                            points edges to be
    
        Returns:
         bl (tuple): x, y of bottom left corner with edges of expansion meters
         tr (tuple): x, y of top right corner with edges of expansion meters
    """
    bl = list(points[0])
    tr = list(points[1])
    inproj = Proj('epsg:4326')
    outproj_code = calculate_epsg(bl)
    outproj = Proj('epsg:' + str(outproj_code))
    bl_utm =  transform(inproj, outproj, bl[1], bl[0])
    tr_utm =  transform(inproj, outproj, tr[1], tr[0])

    distance1 = tr_utm[0] - bl_utm[0]
    distance2 = tr_utm[1] - bl_utm[1]
    expansion1 = (expansion - distance1)/2
    expansion2 = (expansion - distance2)/2
        
    bl_utm = [bl_utm[0] - expansion1, bl_utm[1] - expansion2]
    tr_utm = [tr_utm[0] + expansion1, tr_utm[1] + expansion2]


    zone = str(outproj_code)[3:]
    zone = zone[1:] if zone[0] == "0" else zone
    direction = 'N' if tr[1] >= 0 else 'S'
    utm_epsg = "UTM_" + zone + direction
    return (bl_utm, tr_utm), CRS[utm_epsg]

# Data download

## Cloud and cloud shadow

In [9]:
def identify_clouds(bbox: List[Tuple[float, float]], epsg: 'CRS', time: dict = TIME):
    """ Downloads and calculates cloud cover and shadow
        
        Parameters:
         bbox (list): output of calc_bbox
         epsg (float): EPSG associated with bbox 
         time (tuple): YY-MM-DD - YY-MM-DD bounds for downloading 
    
        Returns:
         cloud_img (np.array): (X, 96, 96) array of cloud probs
         shadows (np.array):  (X, 96, 96) array of shadow binary
         clean_steps (np.array): (N,) array of clean idx
         cloud_dates (np.array): (N,) array of clean cloud datets
    """
    box = BBox(bbox, crs = epsg)
    cloud_request = WcsRequest(
        layer='CLOUD_NEW',
        bbox=box, time=time,
        resx='160m', resy='160m',
        image_format = MimeType.TIFF_d8,
        maxcc=0.75, instance_id=API_KEY,
        custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
        time_difference=datetime.timedelta(hours=96))

    shadow_request = WcsRequest(
        layer='SHADOW',
        bbox=box, time=time,
        resx='20m', resy='20m',
        image_format =  MimeType.TIFF_d16,
        maxcc=0.75, instance_id=API_KEY,
        custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
        time_difference=datetime.timedelta(hours=96))

    cloud_img = np.array(cloud_request.get_data())
    print(f"Original cloud image max is {np.max(cloud_img)}")
    if np.max(cloud_img > 1):
        cloud_img = cloud_img / 255
    assert np.max(cloud_img) <= 1.
    print(f"Cloud_probs shape: {cloud_img.shape}")

    cloud_img = resize(cloud_img, (cloud_img.shape[0], 96, 96), order = 0)
    n_cloud_px = np.sum(cloud_img > 0.33, axis = (1, 2))
    cloud_steps = np.argwhere(n_cloud_px > (96**2 * 0.15))
    clean_steps = [x for x in range(cloud_img.shape[0]) if x not in cloud_steps]
    
    cloud_dates = []
    for date in cloud_request.get_dates():
        if date.year == YEAR - 1:
            cloud_dates.append(-365 + starting_days[(date.month-1)] + date.day)
        if date.year == YEAR:
            cloud_dates.append(starting_days[(date.month-1)] + date.day)
        if date.year == YEAR + 1:
            cloud_dates.append(365 + starting_days[(date.month-1)]+date.day)
    cloud_dates = [val for idx, val in enumerate(cloud_dates) if idx in clean_steps]
    
    shadow_dates = []
    for date in shadow_request.get_dates():
        if date.year == YEAR - 1:
            shadow_dates.append(-365 + starting_days[(date.month-1)] + date.day)
        if date.year == YEAR:
            shadow_dates.append(starting_days[(date.month-1)] + date.day)
        if date.year == YEAR + 1:
            shadow_dates.append(365 + starting_days[(date.month-1)]+date.day)
    shadow_steps = [idx for idx, val in enumerate(shadow_dates) if val in cloud_dates]
    
    shadow_img = np.array(shadow_request.get_data(data_filter = shadow_steps))
    shadow_pus = (shadow_img.shape[1]*shadow_img.shape[2])/(512*512) * shadow_img.shape[0]
    shadow_img = resize(shadow_img, (shadow_img.shape[0], 96, 96, shadow_img.shape[-1]), order = 0)
    
    if np.max(shadow_img > 10):
        print(f"The max shadows is {np.max(shadow_img)}")
        shadow_img = shadow_img / 65535

    cloud_img = np.delete(cloud_img, cloud_steps, 0)
    assert shadow_img.shape[0] == cloud_img.shape[0], (shadow_img.shape, cloud_img.shape)
    shadows = mcm_shadow_mask(np.array(shadow_img), cloud_img) # Make usre this makes sense??
    print(f"Shadows ({shadows.shape}) used {round(shadow_pus, 1)} processing units")
    return cloud_img, shadows, clean_steps, np.array(cloud_dates)

# DEM and slope

In [10]:
def download_dem(plot_id: int, df: 'DataFrame', epsg: 'CRS') -> (np.ndarray, np.ndarray):
    """ Downloads MapZen digital elevation model and return slope

        Parameters:
         plot_id (tuple): plot id from collect earth online (CEO)
         df (pandas.DataFrame): data associated with plot_id from CEO
         epsg (int): UTM EPSG associated with plot_id
    
        Returns:
         slope (arr): (X, Y, 1) array of per-pixel slope from [0, 1]
    """
    location = calc_bbox(plot_id, df = df)
    bbox, epsg = bounding_box(location, expansion = (32+2)*10)
    box = BBox(bbox, crs = epsg)
    dem_request = WcsRequest(data_source=DataSource.DEM,
                         layer='DEM', bbox=box,
                         resx = "10m", resy = "10m",
                         instance_id=API_KEY,
                         image_format= MimeType.TIFF_d32f,
                         custom_url_params={CustomUrlParam.SHOWLOGO: False})
    dem_image_init = dem_request.get_data()[0]
    dem_image = np.copy(dem_image_init)
    dem_image = median_filter(dem_image_init, size = 5)
    slope = calcSlope(dem_image.reshape((1, 32+2, 32+2)),
                      np.full((32+2, 32+2), 10),
                      np.full((32+2, 32+2), 10), 
                      zScale = 1, minSlope = 0.02)
    slope = slope / 90
    slope = slope.reshape((32+2, 32+2, 1))
    slope = slope[1:32+1, 1:32+1, :]
    return slope, dem_image_init

## 10 and 20 meter L2A bands

In [11]:
def download_layer(bbox: List[Tuple[float, float]],
                   clean_steps: np.ndarray, epsg: 'CRS',
                   dates: dict = TIME, year: int = YEAR) -> (np.ndarray, np.ndarray):
    """ Downloads the L2A sentinel layer with 10 and 20 meter bands
        
        Parameters:
         bbox (list): output of calc_bbox
         epsg (float): EPSG associated with bbox 
         time (tuple): YY-MM-DD - YY-MM-DD bounds for downloading 
    
        Returns:
         img (arr):
         img_request (obj): 
    """
    try:
        box = BBox(bbox, crs = epsg)
        image_request = WcsRequest(
                layer='L2A20',
                bbox=box, time=dates,
                image_format = MimeType.TIFF_d16,
                data_source = DataSource.SENTINEL2_L2A,
                maxcc=0.75,
                resx='20m', resy='20m',
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'NEAREST',
                                    constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
                time_difference=datetime.timedelta(hours=96),
            )
        
        image_dates = []
        for date in image_request.get_dates():
            if date.year == YEAR - 1:
                image_dates.append(-365 + starting_days[(date.month-1)] + date.day)
            if date.year == YEAR:
                image_dates.append(starting_days[(date.month-1)] + date.day)
            if date.year == YEAR + 1:
                image_dates.append(365 + starting_days[(date.month-1)]+date.day)
        
        steps_to_download = [i for i, val in enumerate(image_dates) if val in clean_steps]
        dates_to_download = [val for i, val in enumerate(image_dates) if val in clean_steps]
        print(f"The cloud-free image dates are {dates_to_download}")
              
        img_bands = image_request.get_data(data_filter = steps_to_download)
        img_20 = np.stack(img_bands)
        if np.max(img_20) >= 10:
            img_20 = img_20 / 65535
        assert np.max(img_20) <= 2.

        s2_20_usage = (img_20.shape[1]*img_20.shape[2])/(512*512) * (6/3) * img_20.shape[0]
        print(f"Original 20 meter bands size: {img_20.shape}, using {s2_20_usage} PU")
        img_20 = resize(img_20, (img_20.shape[0], IMSIZE, IMSIZE, img_20.shape[-1]), order = 0)
        
        image_request = WcsRequest(
                layer='L2A10',
                bbox=box, time=dates,
                image_format = MimeType.TIFF_d16,
                data_source = DataSource.SENTINEL2_L2A,
                maxcc=0.75,
                resx='10m', resy='10m',
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'BICUBIC',
                                    constants.CustomUrlParam.UPSAMPLING: 'BICUBIC'},
                time_difference=datetime.timedelta(hours=96),
        )
        
        img_bands = image_request.get_data(data_filter = steps_to_download)
        img_10 = np.stack(img_bands)
        print(f"The original L2A image size is: {img_10.shape}")
        img_10 = resize(img_10, (img_10.shape[0], IMSIZE, IMSIZE, img_10.shape[-1]), order = 0)
        img = np.concatenate([img_10, img_20], axis = -1)

        if np.max(img_10) >= 10:
            img_10 = img_10 / 65535
        assert np.max(img_10) <= 2.
        return img, np.array(dates_to_download)

    except Exception as e:
        logging.fatal(e, exc_info=True)

# Super resolution

In [12]:
MDL_PATH = "../src/dsen2/models/"

input_shape = ((4, None, None), (6, None, None))
model = s2model(input_shape, num_layers=6, feature_size=128)
predict_file = MDL_PATH+'s2_032_lr_1e-04.hdf5'
model.load_weights(predict_file)

Instructions for updating:
Colocations handled automatically by placer.


# Download function

In [13]:
def download_new_dem(data_location: 'os.Path',
                     output_folder: 'os.Path',
                     image_format: 'MimeType' = MimeType.TIFF_d16):
    """ Downloads and saves DEM and slope files
        
        Parameters:
         data_location (os.path): 
         output_folder (os.path): 
         image_format (MimeType): 
    
        Returns:
         None
    """
    
    df = pd.read_csv(data_location)
    df.columns = [x.upper() for x in df.columns]
    for column in ['IMAGERY_TITLE', 'STACKINGPROFILEDG', 'PL_PLOTID', 'IMAGERYYEARDG',
                  'IMAGERYMONTHPLANET', 'IMAGERYYEARPLANET', 'IMAGERYDATESECUREWATCH',
                  'IMAGERYENDDATESECUREWATCH', 'IMAGERYFEATUREPROFILESECUREWATCH',
                  'IMAGERYSTARTDATESECUREWATCH','IMAGERY_ATTRIBUTIONS',
                  'SAMPLE_GEOM']:
        if column in df.columns:
            df = df.drop(column, axis = 1)
    
    df = df.dropna(axis = 0)
    plot_ids = sorted(df['PLOT_ID'].unique())
    existing = [int(x[:-4]) for x in os.listdir(output_folder) if ".DS" not in x]
    to_download = [x for x in plot_ids if x not in existing]
    print(f"Starting download of {len(to_download)}"
          f" plots from {data_location} to {output_folder}")
    errors = []
    for i, val in enumerate(to_download):
        print(f"Downloading {i + 1}/{len(to_download)}, {val}")
        initial_bbx = calc_bbox(val, df = df)
        dem_bbx, epsg = bounding_box(initial_bbx, expansion = 32*10)
        slope, dem = download_dem(val, epsg = epsg, df = df)
        print(dem.shape)
        np.save(output_folder + str(val), dem)
        np.save("../data/train-slope/" + str(val), slope)

In [14]:
from scipy.ndimage import median_filter
for i in reversed(os.listdir("../data/train-csv/")):
    if "ethiopia" in i:
    #if ".csv" in i:
        #if any(x in i for x in ["africa-west", "cameroon", "koure", "niger"]):
        tile = download_new_dem("../data/train-csv/" + i, "../data/train-dem/", image_format = MimeType.TIFF_d16)

Starting download of 0 plots from ../data/train-csv/ceo-ethiopia-finetune-sample-data.csv to ../data/train-dem/


In [20]:
%run ../src/io/upload.py
uploader = FileUploader(awskey = AWSKEY, awssecret = AWSSECRET)

In [24]:
def download_plots(data_location, output_folder, image_format = MimeType.TIFF_d16):
    """ Downloads slope and sentinel-2 data for all plots associated
        with an input CSV from a collect earth online survey
        
        Parameters:
         data_location (os.path)
         output_folder (os.path)
        
        Creates:
         output_folder/{plot_id}.npy
    
        Returns:
         None
    """
    df = pd.read_csv(data_location)
    df.columns = [x.upper() for x in df.columns]
    for column in ['IMAGERY_TITLE', 'STACKINGPROFILEDG', 'PL_PLOTID', 'IMAGERYYEARDG',
                  'IMAGERYMONTHPLANET', 'IMAGERYYEARPLANET', 'IMAGERYDATESECUREWATCH',
                  'IMAGERYENDDATESECUREWATCH', 'IMAGERYFEATUREPROFILESECUREWATCH',
                  'IMAGERYSTARTDATESECUREWATCH','IMAGERY_ATTRIBUTIONS',
                  'SAMPLE_GEOM']:
        if column in df.columns:
            df = df.drop(column, axis = 1)
    
    df = df.dropna(axis = 0)
    plot_ids = sorted(df['PLOT_ID'].unique())
    existing = [int(x[:-4]) for x in os.listdir(output_folder) if ".DS" not in x]
    to_download = [x for x in plot_ids if x not in existing]
    
    print(f"Starting download of {len(to_download)}"
          f" plots from {data_location} to {output_folder}")

    for i, val in enumerate(to_download):
        print(f"Downloading {i + 1}/{len(to_download)}, {val}")
        initial_bbx = calc_bbox(val, df = df)
        sentinel2_bbx, epsg = bounding_box(initial_bbx, expansion = IMSIZE*10)
        cloud_bbx, _ = bounding_box(initial_bbx, expansion = 96*10)
        try:
            # Identify cloud steps, download DEM, and download L2A series
            cloud_probs, shadows, clean_steps, image_dates = identify_clouds(cloud_bbx, epsg = epsg)
            dem, _ = download_dem(val, epsg = epsg, df = df)
            to_remove, _ = calculate_cloud_steps(cloud_probs, image_dates)
            cloud_probs = np.delete(cloud_probs, to_remove, 0)
            clean_dates = np.delete(image_dates, to_remove)
            shadows = np.delete(shadows, to_remove, 0)
            s2, s2_dates = download_layer(sentinel2_bbx, clean_steps = clean_dates, epsg = epsg)    
            
            # Step to ensure that shadows, clouds, sentinel l2a have aligned dates
            print(f"Shadows {shadows.shape}, clouds {cloud_probs.shape}, S2, {s2.shape}, S2d, {s2_dates.shape}")
            to_remove_clouds = [i for i, val in enumerate(clean_dates) if val not in s2_dates]
            if len(to_remove_clouds) >= 1:
                cloud_probs = np.delete(cloud_probs, to_remove_clouds, 0)
                shadows = np.delete(shadows, to_remove_clouds, 0)
            print(f"Shadows {shadows.shape}, clouds {cloud_probs.shape}, S2, {s2.shape}, S2d, {s2_dates.shape}")

            
            to_remove = remove_missed_clouds(s2)
            s2 = np.delete(s2, to_remove, 0)
            cloud_probs = np.delete(cloud_probs, to_remove, 0)
            s2_dates = np.delete(s2_dates, to_remove)
            shadows = np.delete(shadows, to_remove, 0)
            print(f"Removing {len(to_remove)} steps based on ratio")
            
            cloud_probs = cloud_probs[:, 24:-24, 24:-24]
            shadows = shadows[:, 24:-24, 24:-24]
            x, interp = remove_cloud_and_shadows(s2, cloud_probs, shadows, s2_dates)
            to_remove = np.argwhere(np.mean(interp, axis = (1, 2, 3)) > 0.5)
            x = np.delete(x, to_remove, 0)
            cloud_probs = np.delete(cloud_probs, to_remove, 0)
            s2_dates = np.delete(s2_dates, to_remove)
            shadows = np.delete(shadows, to_remove, 0)
            print(f"Removing {len(to_remove)} steps with >50% interpolation")
            
            x_to_save = np.copy(x)
            x_to_save = np.clip(x_to_save, 0, 1)
            x_to_save = np.trunc(x_to_save * 65535).astype(np.uint16)
            np.save(f"../data/train-raw/{str(val)}", x_to_save)
            file = f"../data/train-raw/{str(val)}.npy"
            key = f'restoration-mapper/model-data/train/raw/{str(val)}.npy'
            uploader.upload(bucket = 'restoration-monitoring', key = key, file = file)
            
            x = x[:, 8:40, 8:40, :]
            x = superresolve(x, model)
            x = x[:, 8:24, 8:24, :]
            dem = np.tile(dem.reshape((1, 32, 32, 1)), (x.shape[0], 1, 1, 1))
            dem = dem[:, 8:-8, 8:-8, :]
            x = np.concatenate([x, dem], axis = -1)
            x[:, :, :, -1] /= 90
            print(f"Shape after DEM: {x.shape}")

            # Calculate indices
            tiles = evi(x, True)
            tiles = bi(tiles, True)
            tiles = msavi2(tiles, True)
            x = si(tiles, True)
            
            for band in range(0, 15):
                for time in range(0, x.shape[0]):
                    x_i = x[time, :, :, band]
                    x_i[np.argwhere(np.isnan(x_i))] = np.mean(x_i)
                    x[time, :, :, band] = x_i

            # Interpolate linearly to 5 day frequency
            tiles, max_distance = calculate_and_save_best_images(x, s2_dates)
            sm = Smoother(lmbd = 800, size = tiles.shape[0], nbands = 14, dim = 16)
            tiles = sm.interpolate_array(tiles)
            print(f"There are {np.sum(np.isnan(tiles))} NA values")
            
            if max_distance <= 240:
                np.save(output_folder + str(val), tiles)
                print(f"Saved array of {tiles.shape} shape to {val} \n")
            else:
                print(f"Skipping {val} because there is a {max_distance} distance \n")

        except Exception as e:
            print(e)
            logging.fatal(e, exc_info=True)

In [ ]:
for i in reversed(os.listdir("../data/train-csv/")):
    if ".csv" in i:
    #if ".csv" in i:
        #if any(x in i for x in ["africa-west", "cameroon", "koure", "niger"]):
        tile = download_plots("../data/train-csv/" + i, "../data/train-s2-new/", image_format = MimeType.TIFF_d16)

Starting download of 0 plots from ../data/train-csv/koure-train.csv to ../data/train-s2-new/
Starting download of 0 plots from ../data/train-csv/ceo-brazil-sao-paulo-2-sample-data-2020-07-29.csv to ../data/train-s2-new/
Starting download of 0 plots from ../data/train-csv/australia-train.csv to ../data/train-s2-new/
Starting download of 0 plots from ../data/train-csv/kenya-farm-train.csv to ../data/train-s2-new/
Starting download of 0 plots from ../data/train-csv/cameroon-finetune.csv to ../data/train-s2-new/
Starting download of 0 plots from ../data/train-csv/ghana-kwofu-small-train.csv to ../data/train-s2-new/
Starting download of 0 plots from ../data/train-csv/ghana-mid-train.csv to ../data/train-s2-new/
Starting download of 28 plots from ../data/train-csv/lac-train.csv to ../data/train-s2-new/
Original cloud image max is 255
Cloud_probs shape: (69, 6, 6)



Shadows ((51, 96, 96)) used 0.4 processing units
0, Good steps: [ 7 12 22 27], min dist of 10, and 0.01 thresh
1, Good steps: [32 37 42 52 57], min dist of 20, and 0.01 thresh
2, Good steps: [62 67 72 77 82 87], min dist of 20, and 0.01 thresh
3, Good steps: [ 92  97 102 107 112], min dist of 65, and 0.06 thresh
4, Good steps: [], min dist of 365, and 0.15 thresh
5, Good steps: [157 177], min dist of 20, and 0.01 thresh
6, Good steps: [187 197 207], min dist of 10, and 0.01 thresh
7, Good steps: [212], min dist of 50, and 0.03 thresh
8, Good steps: [262], min dist of 50, and 0.03 thresh
9, Good steps: [287 302], min dist of 15, and 0.01 thresh
10, Good steps: [307 312 317 332], min dist of 5, and 0.01 thresh
11, Good steps: [342 347 352 357], min dist of 5, and 0.01 thresh
[ 0  1  2  3  4  5 26 30 45 46 47 48 49 50]
[0.30555556 0.         0.         0.         0.         0.
 0.25       0.02777778 0.         0.         0.         0.
 0.         0.        ]
The cloud-free image dates ar


Shadows ((49, 96, 96)) used 0.4 processing units
0, Good steps: [-12  13  18  23], min dist of 15, and 0.01 thresh
1, Good steps: [38 48 58], min dist of 20, and 0.01 thresh
2, Good steps: [63 68 83], min dist of 25, and 0.01 thresh
3, Good steps: [103 108], min dist of 20, and 0.01 thresh
4, Good steps: [128 133 138], min dist of 25, and 0.01 thresh
5, Good steps: [158], min dist of 30, and 0.03 thresh
6, Good steps: [183 208], min dist of 25, and 0.01 thresh
7, Good steps: [213 228 238], min dist of 25, and 0.01 thresh
8, Good steps: [253 258 263], min dist of 15, and 0.01 thresh
9, Good steps: [283 293], min dist of 20, and 0.01 thresh
10, Good steps: [313 323 328], min dist of 20, and 0.01 thresh
11, Good steps: [338 343 363 373], min dist of 5, and 0.01 thresh
[ 0  1  3  4  8 11 16 23 25 26 29 31 41 44 47 48]
[0.         0.         0.02777778 0.05555556 0.02777778 0.05555556
 0.02777778 0.25       0.05555556 0.02777778 0.16666667 0.13888889
 0.05555556 0.11111111 0.         0.   


Shadows ((62, 96, 96)) used 0.5 processing units
0, Good steps: [-6 -1  4 29], min dist of 5, and 0.01 thresh
1, Good steps: [34 39 44 49 54], min dist of 15, and 0.01 thresh
2, Good steps: [59 64 69 79 84 89], min dist of 25, and 0.01 thresh
3, Good steps: [ 94 104 109 114 119], min dist of 20, and 0.01 thresh
4, Good steps: [124 129 134 144], min dist of 25, and 0.01 thresh
5, Good steps: [159 169 179], min dist of 25, and 0.01 thresh
6, Good steps: [184 209], min dist of 30, and 0.03 thresh
7, Good steps: [214 219 224 229 239], min dist of 20, and 0.01 thresh
8, Good steps: [244 249 259 264], min dist of 20, and 0.01 thresh
9, Good steps: [279 284 294], min dist of 25, and 0.01 thresh
10, Good steps: [309 314 324], min dist of 20, and 0.01 thresh
11, Good steps: [334 339 344 349], min dist of 10, and 0.01 thresh
[ 0  1  2  6 20 53 54 55 56 57 58 59 60 61]
[0.         0.         0.         0.11111111 0.05555556 0.
 0.         0.         0.         0.         0.05555556 0.
 0.       


Shadows ((38, 96, 96)) used 0.3 processing units
0, Good steps: [13 23 28], min dist of 10, and 0.01 thresh
1, Good steps: [38 58], min dist of 25, and 0.01 thresh
2, Good steps: [63 68 83], min dist of 25, and 0.01 thresh
3, Good steps: [103], min dist of 40, and 0.03 thresh
4, Good steps: [138 143 148], min dist of 45, and 0.03 thresh
5, Good steps: [168], min dist of 25, and 0.01 thresh
6, Good steps: [183 188 193 203], min dist of 15, and 0.01 thresh
7, Good steps: [218 223 238], min dist of 10, and 0.01 thresh
8, Good steps: [248], min dist of 15, and 0.01 thresh
9, Good steps: [273 298 303], min dist of 25, and 0.01 thresh
10, Good steps: [], min dist of 365, and 0.15 thresh
11, Good steps: [338 343 388], min dist of 5, and 0.01 thresh
[ 1  5  6  7 11 13 23 26 29 35 36]
[0.02777778 0.05555556 0.11111111 0.08333333 0.02777778 0.05555556
 0.08333333 0.08333333 0.05555556 0.13888889 0.05555556]
The cloud-free image dates are [13, 23, 28, 38, 58, 68, 103, 138, 143, 148, 168, 183, 18

...


Shadows ((46, 96, 96)) used 0.4 processing units
0, Good steps: [-12  23  28], min dist of 10, and 0.01 thresh
1, Good steps: [38 43 48 53 58], min dist of 20, and 0.01 thresh
2, Good steps: [63 68 73 83 88], min dist of 25, and 0.01 thresh
3, Good steps: [ 98 103 113], min dist of 30, and 0.03 thresh
4, Good steps: [128 138 143], min dist of 15, and 0.01 thresh
5, Good steps: [163 168 178], min dist of 15, and 0.01 thresh
6, Good steps: [183 188 193], min dist of 5, and 0.01 thresh
7, Good steps: [218 223 238], min dist of 15, and 0.01 thresh
8, Good steps: [248 263], min dist of 15, and 0.01 thresh
9, Good steps: [303], min dist of 40, and 0.03 thresh
10, Good steps: [323 328], min dist of 10, and 0.15 thresh
11, Good steps: [338 343 363 373], min dist of 10, and 0.01 thresh
[ 1  2  3  4  5 20 31 34 45]
[0.08333333 0.05555556 0.08333333 0.02777778 0.08333333 0.05555556
 0.16666667 0.22222222 0.        ]
The cloud-free image dates are [-12, 23, 28, 38, 43, 48, 53, 58, 68, 73, 88, 98,


Shadows ((27, 96, 96)) used 0.2 processing units
0, Good steps: [-29  -9], min dist of 60, and 0.06 thresh
1, Good steps: [46 51], min dist of 45, and 0.03 thresh
2, Good steps: [61], min dist of 40, and 0.06 thresh
3, Good steps: [ 96 101 111], min dist of 65, and 0.06 thresh
4, Good steps: [], min dist of 365, and 0.15 thresh
5, Good steps: [156], min dist of 45, and 0.03 thresh
6, Good steps: [196 211], min dist of 20, and 0.03 thresh
7, Good steps: [236], min dist of 25, and 0.01 thresh
8, Good steps: [261], min dist of 25, and 0.03 thresh
9, Good steps: [276 301], min dist of 25, and 0.01 thresh
10, Good steps: [311 326], min dist of 15, and 0.06 thresh
11, Good steps: [341 381 396], min dist of 15, and 0.15 thresh
[ 2  3  4 11 13 19 23]
[0.11111111 0.11111111 0.13888889 0.16666667 0.13888889 0.22222222
 0.13888889]
The cloud-free image dates are [-29, -9, 46, 51, 61, 96, 101, 111, 156, 196, 211, 236, 261, 276, 301, 311, 326, 341, 381, 396]
Original 20 meter bands size: (20, 24, 


Shadows ((36, 96, 96)) used 0.3 processing units
0, Good steps: [-30   0  30], min dist of 5, and 0.01 thresh
1, Good steps: [35 50 55], min dist of 20, and 0.01 thresh
2, Good steps: [60 75 85], min dist of 20, and 0.01 thresh
3, Good steps: [ 90 110], min dist of 35, and 0.03 thresh
4, Good steps: [125 145], min dist of 20, and 0.03 thresh
5, Good steps: [155 160 175], min dist of 20, and 0.01 thresh
6, Good steps: [185 195 205], min dist of 30, and 0.03 thresh
7, Good steps: [215 235 240], min dist of 85, and 0.06 thresh
8, Good steps: [], min dist of 365, and 0.15 thresh
9, Good steps: [285], min dist of 50, and 0.03 thresh
10, Good steps: [310 325], min dist of 25, and 0.01 thresh
11, Good steps: [335 340 350 360], min dist of 5, and 0.01 thresh
[ 2  8 13 16 33 34 35]
[0.05555556 0.02777778 0.22222222 0.08333333 0.11111111 0.
 0.02777778]
The cloud-free image dates are [-30, 0, 30, 35, 50, 55, 60, 75, 90, 110, 125, 145, 155, 160, 175, 185, 195, 205, 215, 235, 240, 285, 310, 325, 


Shadows ((49, 96, 96)) used 0.4 processing units
0, Good steps: [13 18 23 28], min dist of 15, and 0.01 thresh
1, Good steps: [38 43 48 53 58], min dist of 15, and 0.01 thresh
2, Good steps: [63 68 73 83 88], min dist of 25, and 0.01 thresh
3, Good steps: [ 98 103], min dist of 10, and 0.01 thresh
4, Good steps: [123 138 143 148], min dist of 5, and 0.03 thresh
5, Good steps: [168], min dist of 20, and 0.03 thresh
6, Good steps: [183 193], min dist of 15, and 0.01 thresh
7, Good steps: [218 223 238], min dist of 10, and 0.03 thresh
8, Good steps: [248 263], min dist of 15, and 0.01 thresh
9, Good steps: [298 303], min dist of 20, and 0.01 thresh
10, Good steps: [323], min dist of 20, and 0.06 thresh
11, Good steps: [338 343 373 388], min dist of 15, and 0.01 thresh
[ 0  1  2  3  4 21 22 24 30 32 37 39 45 46]
[0.         0.25       0.11111111 0.13888889 0.11111111 0.16666667
 0.13888889 0.22222222 0.02777778 0.11111111 0.25       0.05555556
 0.22222222 0.02777778]
The cloud-free image 


Shadows ((14, 96, 96)) used 0.1 processing units
0, Good steps: [-30  10], min dist of 30, and 0.03 thresh
1, Good steps: [40 45], min dist of 55, and 0.03 thresh
2, Good steps: [], min dist of 365, and 0.15 thresh
3, Good steps: [100 110], min dist of 80, and 0.06 thresh
4, Good steps: [], min dist of 365, and 0.15 thresh
5, Good steps: [], min dist of 365, and 0.15 thresh
6, Good steps: [190], min dist of 95, and 0.15 thresh
7, Good steps: [], min dist of 365, and 0.15 thresh
8, Good steps: [], min dist of 365, and 0.15 thresh
9, Good steps: [275 285], min dist of 85, and 0.1 thresh
10, Good steps: [320], min dist of 40, and 0.03 thresh
11, Good steps: [335 375], min dist of 15, and 0.01 thresh
[ 1 12]
[0.08333333 0.02777778]
The cloud-free image dates are [-30, 10, 40, 45, 100, 110, 190, 275, 285, 320, 335, 375]
Original 20 meter bands size: (12, 24, 24, 6), using 0.052734375 PU
The original L2A image size is: (12, 48, 48, 4)
Shadows (12, 96, 96), clouds (12, 96, 96), S2, (12, 48, 